In [1]:
with open(file='/kaggle/input/moby-dick-herman-melville/melville-moby_dick.txt', mode='r', encoding='utf-8', ) as input_fp:
    data = input_fp.read()

Our text is oddly formatted; we need to parse it into sentences to use gensim. So we use a sentence parser from spacy.

In [2]:
from arrow import now
from spacy import load
time_start = now()
spacy_model = load('en_core_web_sm')
spacy_model.max_length = 1200000
spacy_result = spacy_model(data.replace('\n', ' '))
documents = [item.text for item in spacy_result.sents]
print(now() - time_start)

0:00:52.893811


In [3]:
from gensim.corpora import Dictionary
from gensim.parsing.preprocessing import preprocess_documents
texts = preprocess_documents(docs=documents)
dictionary = Dictionary(texts)
dictionary.save('/kaggle/working/melville.dict')
print(dictionary)

Dictionary<10228 unique tokens: ['chapter', 'loom', 'ishmael', 'ago', 'have']...>


In [4]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [5]:
from gensim.models import HdpModel
time_start = now()
hdp_model = HdpModel(corpus=corpus, id2word=dictionary, random_state=2023)
print(now() - time_start)
print('topic count: {}'.format(len(hdp_model.get_topics())))

0:00:14.238503
topic count: 150


In [6]:
from gensim.models import Word2Vec
time_start = now()
word2vec_model = Word2Vec(sentences=texts, vector_size=100, window=5, min_count=1, workers=4)
print(now() - time_start)

0:00:00.675471


In [7]:
from sklearn.manifold import TSNE
time_start = now()
tsne = TSNE(random_state=2023)
tsne_result = tsne.fit_transform(X=word2vec_model.wv.vectors)
print(now() - time_start)

0:01:06.811801


In [8]:
import pandas as pd
from plotly.express import scatter
scatter(data_frame=pd.DataFrame(data=tsne_result, columns=['x', 'y']), x='x', y='y')